## 监督学习和无监督学习
本书中涉及的监督学习算法如下：
- 神经网络
- 决策树
- 支持向量机
- 贝叶斯过滤

涉及到的非监督学习算法如下：
- 聚类
- 非负矩阵因式分解
- 自组织映射

本节主要内容即为聚类

## 单词向量
我们需要一组指定的词汇在每个博客订阅源中出现的次数

为此，我们需要下载一系列博客订阅源，从中提取文本，建立一个单词频度表

通过`pip install feedparser`下载Universal Feed Parser这个RSS文档解析器。

In [1]:
import feedparser
import re

# 返回一个RSS订阅源的标题和包含单词计数情况的字典
def getwordcounts(url):
    # 解析订阅源
    d=feedparser.parse(url)
    wc={}
    
    # 循环遍历所有的文章条目
    for e in d.entries:
        if 'summary' in e:
            summary=e.summary
        else:
            summary=e.description
            
        # 提取一个单词列表
        words=getwords(e.title+' '+summary)
        for word in words:
            wc.setdefault(word,0)
            wc[word]+=1
    return d.feed.title,wc

# 将摘要传给函数getwords,把其中所有的HTML标记剥离掉，并以非字母字符作为分隔符拆分出单词，再将结果以列表的形式加以返回
def getwords(html):
    # 去除所有的HTML标记
    txt=re.compile(r'<[^>]+>').sub('',html)
    
    #利用所有非字母字符拆分出单词
    words=re.compile(r'[^A-Z^a-z]+').split(txt)
    
    #转化成小写形式
    return [word.lower() for word in words if words!='']

循环遍历订阅源（在本地文件夹下）并生成数据集。

In [2]:
apcount={}
wordcounts={}
with open('feedlist.txt','r') as f:
    feedlist=[line for line in f.readlines()]
#  feedlist=[line for line in file('feedlist.txt')]   
# 生成每个博客的单词统计
for feedurl in feedlist:
    try:
        title,wc=getwordcounts(feedurl)
        wordcounts[title]=wc
        for word,count in wc.items():
            apcount.setdefault(word,0)
            if count>1:
                apcount[word]+=1
    except:
        print('Failed to parse feed %s' % feedurl)

Failed to parse feed http://battellemedia.com/index.xml

Failed to parse feed http://blog.guykawasaki.com/index.rdf

Failed to parse feed http://feeds.searchenginewatch.com/sewblog

Failed to parse feed http://blog.topix.net/index.rdf

Failed to parse feed http://blogs.abcnews.com/theblotter/index.rdf

Failed to parse feed http://feeds.feedburner.com/ConsumingExperienceFull

Failed to parse feed http://flagrantdisregard.com/index.php/feed/

Failed to parse feed http://featured.gigaom.com/feed/

Failed to parse feed http://googleblog.blogspot.com/rss.xml

Failed to parse feed http://headrush.typepad.com/creating_passionate_users/index.rdf

Failed to parse feed http://feeds.feedburner.com/instapundit/main

Failed to parse feed http://jeremy.zawodny.com/blog/rss2.xml

Failed to parse feed http://michellemalkin.com/index.rdf

Failed to parse feed http://moblogsmoproblems.blogspot.com/rss.xml

Failed to parse feed http://beta.blogger.com/feeds/27154654/posts/full?alt=rss

Failed to parse fe

建立一个单词列表。去掉类似“the”这样的单词和“film-flam”这样的生僻单词。

我们可以选择10%-50%出现的单词。

In [3]:
wordlist=[]
for w,bc in apcount.items():
    frac=float(bc)/len(feedlist)
    if frac>0.1 and frac<0.5:
        wordlist.append(w)

利用上述单词列表和博客列表来建立一个文本文件，其中包含一个大矩阵，记录着针对每个博客的所有单词的统计情况

In [4]:
with open('blogdata.txt','w') as f:
    f.write('Blog')
    for word in wordlist:
        f.write('\t%s' %word)
    f.write('\n')
    for blog,wc in wordcounts.items():
        f.write(blog)
        for word in wordlist:
            if word in wc:
                f.write('\t%d' %wc[word])
            else:
                f.write('\t0')
        f.write('\n')

这个时候目录下应该会有一个blogdata.txt文件，我们来看一下（格式比较乱）

In [5]:
with open('blogdata.txt','r') as f:
    print(f.read())

Blog	close	ever	blog	without	list	part	nbsp	want	things	got	job	or	lot	words	before	only	has	original	face	i	including	couple	well	asked	was	name	take	book	week	each	top	around	be	u	special	any	off	stuff	content	times	else	deal	so	june	problem	point	ways	amazon	would	example	number	page	can	place	clear	looks	end	car	friends	started	if	create	use	does	gets	real	help	free	idea	may	social	her	told	ve	great	side	action	bad	while	going	available	much	than	were	show	sure	could	year	four	live	saying	probably	most	water	long	another	media	check	maybe	back	p	almost	kind	during	said	latest	possible	look	they	who	youtube	security	turn	over	building	big	when	least	but	google	took	case	go	behind	even	way	thing	used	other	been	keep	summer	month	two	especially	down	here	make	think	good	really	support	email	music	piece	night	put	system	ideas	its	gave	ceo	process	something	change	thought	anything	hours	just	buy	where	six	next	say	means	person	then	c	went	line	service	seen	should	my	last	later	across	hi

## 分级聚类
不断的将最为相似的两个群组两两合并，构造出一个群组的层级结构

每个群组都是从单一元素开始的。在每次迭代的过程中，分级聚类算法会计算每两个群组间的距离，并将距离最近的两个群组合并成一个群组。

下面写一个函数，将数据集中的头一行读入一个代表了列名的列表，将最左边的一列读入了一个代表行名的列表，最后它把剩下所有的数据都放入了一个大列表。

In [11]:
def readfile(filename):
    with open(filename,'r') as f:
        lines=[line for line in f.readlines()]
    
    # 第一行是列标题
    colnames=lines[0].strip().split('\t')[1:]
    rownames=[]
    data=[]
    for line in lines[1:]:
        p=line.strip().split('\t')
        # 每行的第一列是行名
        rownames.append(p[0])
        # 剩余部分就是该行对应的数据
        data.append([float(x) for x in p[1:]])
    return rownames,colnames,data

下面我们定义紧密度。使用**皮尔逊相关度**来判断两组数据与某条直线的拟合程度。


In [75]:
from math import sqrt
def pearson(v1,v2):
    sum1=sum(v1)
    sum2=sum(v2)
    
    #求平方和
    sum1Sq=sum([pow(v,2) for v in v1])
    sum2Sq=sum([pow(v,2) for v in v2])
    
    # 求乘积之和
    n=min(len(v1),len(v2))  # index的错误
    pSum=sum([v1[i]*v2[i] for i in range(n)])
  
    # 计算r
    num=pSum-(sum1*sum2/len(v1))
    den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
    if den==0: 
        return 0

    return 1.0-num/den 

皮尔逊相关度在两者完全匹配的情况下为1.0，毫无关系的情况下为0

而我们用1减去最后的结果，目的是让相似度越大的两个元素之间的距离变得越小

新建一个bicluster类，保存节点的位置信息。

In [76]:
class bicluster:
    def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
        self.left=left
        self.right=right
        self.vec=vec
        self.id=id
        self.distance=distance

分级聚类算法以一组对应于原始数据项的聚类开始。函数的主循环部分会尝试每一组可能的配对并计算它们的相关度，以此找出最佳配对。

最佳配对的两个聚类会被合并成新的聚类。新的聚类中包含的数据，等于两个旧聚类的数据求均值后得到的结果。

这一过程持续下去，直到只剩下一个聚类为止。

这个过程十分耗时，可以将每个配对的相关度计算保存起来。


In [81]:
def hcluster(rows,distance=pearson):
    distances={}
    currentclustid=-1
    
    # 最开始的聚类就是数据集中的行
    clust=[bicluster(rows[i],id=i) for i in range(len(rows))]
    
    while len(clust)>1:
        lowestpair=(0,1)
        closest=distance(clust[0].vec,clust[1].vec)
        
        # 遍历每一个配对，寻找最小距离
        for i in range(len(clust)):
            for j in range(i+1,len(clust)):
                # 用distances来缓存距离的计算值，注意，distance是计算距离的函数
                if(clust[i].id,clust[j].id) not in distances:
                    distances[(clust[i].id,clust[j].id)]=distance(clust[i].vec,clust[j].vec)
                    #distances[(clust[i].id,clust[j].id)]=distance(clust[i],clust[j])
                    
                d=distances[(clust[i].id,clust[j].id)]
                
                if d<closest:
                    closest=d
                    lowerstpair=(i,j)
        # 计算两个聚类的平均值
        length_min=min(len(clust[lowerstpair[0]].vec),len(clust[lowestpair[1]].vec))
        # mergevec=[(clust[lowerstpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 for i in range(len(clust[0].vec))]
        mergevec=[(clust[lowerstpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 for i in range(length_min)]
        
        # 建立新的聚类
        newcluster=bicluster(mergevec,left=clust[lowestpair[0]],right=clust[lowestpair[1]],distance=closest,id=currentclustid)
        
        # 不在原始集合中的聚类，其id为负数
        currentclustid-=1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)
        
    return clust[0]

然后，我们把文件加载进来

In [82]:
blognames,words,data=readfile('blogdata.txt')
clust=hcluster(data)

递归遍历聚类树，并将其以类似文件系统层级结构的形式打印出来。

In [102]:
def printclust(clust,labels=None,n=0):
    # 利用缩进来建立层级布局
    for i in range(n):
        print(' ',end="")
    if clust.id<0:
        # 负数代表这是一个分支
        print('-')
    else:
        # 正数标记代表这是一个叶节点
        if labels==None:
            print(clust.id)
        else:
            print(labels[clust.id])
    
    # 现在开始打印右侧分支和左侧分支
    if clust.left!=None:
        printclust(clust.left,labels=labels,n=n+1)
    if clust.right!=None:
        printclust(clust.right,labels=labels,n=n+1)

In [103]:
printclust(clust,labels=blognames)

-
 -
  -
   -
    -
     Latest from Crooks and Liars
     Lifehack
    -
     Mashable
     Gizmodo
   -
    -
     Gothamist
     Techdirt.
    -
     Boing Boing
     Oilman
  -
   -
    -
     -
      4
      PaulStamatiou.com - Technology, Design and Photography
     -
      Slashdot
      O'Reilly Radar
    -
     -
      blog maverick
      ThinkProgress - Medium
     -
      Deadspin
      ShoeMoney
   -
    -
     -
      The Dish
      456 Berea Street
     -
      TechCrunch
      plasticbag.org
    -
     -
      Schneier on Security
      Engadget RSS Feed
     -
      MetaFilter
      SpikedHumor - Today's Videos and Pictures
 -
  -
   -
    -
     -
      Celebslam
      Search Engine Roundtable
     -
      Neil Gaiman's Journal
      43 Folders
    -
     -
      Kotaku
      Joho the Blog
     -
      TMZ.com
      Joi Ito's Web
   -
    -
     -
      Lifehacker
      PerezHilton
     -
      WIL WHEATON dot NET
      Google Operating System
    -
     -
      The Fu

## 绘制树状图
借助树状图，我们可以更加清晰的理解聚类

首先，需要利用一个函数来返回给定聚类的总体高度

In [104]:
def getheight(clust):
    # 这是一个叶节点吗?若是，则高度为1
    if clust.left==None and clust.right==None:
        return 1
    # 否则，高度为每个分支的高度之和
    return getheight(clust.left)+getheight(clust.right)

In [106]:
def getdepth(clust):
    # 一个叶节点的距离是0.0
    if clust.left==None and clust.right==None:
        return 0
    # 一个枝节点的距离等于左右两侧分支中距离比较大者，加上该枝节点自身的距离
    return max(getdepth(clust.left),getdepth(clust.right)+clust.distance)

函数drawdendgrogram为每一个最终生成的聚类创建一个高度为20像素，像素固定的图片

在根节点出调用drawnode函数，并令其处于整副图片正中间的位置

In [111]:
from PIL import Image,ImageDraw
def drawdendrogram(clust,labels,jpeg='clusters.jpg'):
    # 高度和宽度
    h=getheight(clust)*20
    w=1200
    depth=getdepth(clust)

    # 由于宽度是固定的，因此我们需要对距离值做出相应的调整
    scaling=float(w-150)/depth

    # 新建一个白色背景的图片
    img=Image.new('RGB',(w,h),(255,255,255))
    draw=ImageDraw.Draw(img)

    draw.line((0,h/2,10,h/2),fill=(255,0,0))    

    # 画第一个节点
    drawnode(draw,clust,10,(h/2),scaling,labels)
    img.save(jpeg,'JPEG')

drawnode的功能是接收一个聚类机器位置作为输入参数。函数取到子节点的高度，并计算出这些节点所在的位置，然后用线条将它们连接起来。

In [112]:
def drawnode(draw,clust,x,y,scaling,labels):
    if clust.id<0:
        h1=getheight(clust.left)*20
        h2=getheight(clust.right)*20
        top=y-(h1+h2)/2
        bottom=y+(h1+h2)/2
        # 线的长度
        ll=clust.distance*scaling
   
        draw.line((x,top+h1/2,x,bottom-h2/2),fill=(255,0,0))    
    
        draw.line((x,top+h1/2,x+ll,top+h1/2),fill=(255,0,0))    
    
        draw.line((x,bottom-h2/2,x+ll,bottom-h2/2),fill=(255,0,0))        

        drawnode(draw,clust.left,x+ll,top+h1/2,scaling,labels)
        drawnode(draw,clust.right,x+ll,bottom-h2/2,scaling,labels)
    else:   
        draw.text((x+5,y-7),labels[clust.id],(0,0,0))

In [113]:
drawdendrogram(clust,blognames,jpeg='blogclust.jpg')

## 列聚类
在博客数据集中，列代表的是单词，知道哪些单词时常会结合在一起使用，是非常有意义的。

将整个数据集转置，使列变成行，其中的每一行都对应一组数字，这组数字指明了某个单词再每篇博客中出现的次数。

In [114]:
def rotatematrix(data):
    newdata=[]
    for i in range(len(data[0])):
        newrow=[data[j][i] for j in range(len(data))]
        newdata.append(newrow)
    return newdata

In [116]:
rdata=rotatematrix(data)
wordclust=hcluster(rdata)
drawdendrogram(wordclust,labels=words,jpeg='wordclust.jpg')